In [ ]:
%store -r table_name

Lets take a look at our revised mcp file that we'll be deploying to AgentCore Runtime. Here we've added an `update_booking` method to update bookings.

In [2]:
# code to look at restaurant_mcp_agentcore_rt.py

Now, with the help of agent core took kit, we'll deploy it to agentcore runtime. Be sure to use the table name you specificied in our previous notebook. We'll print here so you can be sure.

In [3]:
print(table_name)

restaurant_bookings


We'll need to create an execution role for this agent in agent core so it can interact with DynamoDB. Let's use our helper function for that?

In [4]:
from utils import create_agent_core_execution_role
agent_core_role_arn = create_agent_core_execution_role(agent_name='restaurant_booking', dynamodb_table_name=table_name)

In [5]:
print(agent_core_role_arn)

arn:aws:iam::228707323172:role/AgentCoreExecutionRole_restaurant_booking


with our agentcore runtime configured, we'll now deploy it. Run the following command in your terminal.
```bash 
agentcore configure -e restaurant_mcp_agentcore_rt_with_agent.py
```

When prompted for an Execution Role in the agentcore configuration process, be sure to enter the above role ARN.

Now lets launch our agent - run the following the same terminal. This will deploy your agent to AgentCore Runtime.
```bash
agentcore launch
```

Now we're ready to test our agent using the agent core runtime:

```bash
agentcore invoke '{"prompt": "can i make a booking for tonight, for 6 people, at 7 PM? Name is Ferdinand."}'
```

Verify that you're returned a valid booking ID.


Let's ask about the details of our recent booking:
```bash
agentcore invoke '{"prompt": "can you tell me the booking details for booking id <booking id>."}'
```

Let's update our booking
```bash
agentcore invoke '{"prompt": "can i update a booking i made? Booking is under the name Ferdinand and I need it for 10 people now."}'
```